In [1]:
###############################################################################
def common_mode_vondrak(Sgts, fc, verbose=True):

    import numpy as np
    import scipy.linalg
    import pyacs.lib.glinalg
    import copy
    from pyacs.gts.Gts import Gts

    def get_A(t):

        import numpy

        eps = (7.223147119819503*fc) ** 6 / (len(t) - 3)
        num = 6 * numpy.sqrt(t[2:-1] - t[1:-2])


        den = numpy.sqrt(t[-1] - t[0])

        a = numpy.hstack((0, 0, 0, num / den / ((t[0:-3] - t[1:-2]) * (t[0:-3] - t[2:-1]) * (t[0:-3] - t[3:])),   0, 0, 0))
        b = numpy.hstack((0, 0, 0, num / den / ((t[1:-2] - t[0:-3]) * (t[1:-2] - t[2:-1]) * (t[1:-2] - t[3:])),   0, 0, 0))
        c = numpy.hstack((0, 0, 0, num / den / ((t[2:-1] - t[0:-3]) * (t[2:-1] - t[1:-2]) * (t[2:-1] - t[3:])),   0, 0, 0))
        d = numpy.hstack((0, 0, 0, num / den / ((t[3:]   - t[0:-3]) * (t[3:]   - t[1:-2]) * (t[3:]   - t[2:-1])), 0, 0, 0))

        d0 = eps + a[3:]**2 + b[2:-1]**2 + c[1:-2]**2 + d[0:-3]**2
        d1 = a[3:-1] * b[3:-1] + b[2:-2] * c[2:-2] + c[1:-3] * d[1:-3]
        d2 = a[3:-2] * c[3:-2] + b[2:-3] * d[2:-3]
        d3 = a[3:-3] * d[3:-3]

        A  = numpy.diag(d0) + numpy.diag(d1,1) + numpy.diag(d1,-1) + numpy.diag(d2,2) + numpy.diag(d2,-2) + numpy.diag(d3,3) + numpy.diag(d3,-3)

        return A/eps

    # convert Sgts to observation tensor
    import pyeq.lib.obs_tensor.sgts2obs_tensor
    import pyacs.lib.astrotime as at
    T_OBS_RAW , np_names_t_obs, np_obs_date_s = \
        pyeq.lib.obs_tensor.sgts2obs_tensor.sgts2tensor( Sgts, rounding='day' , verbose=False )

    print('T_OBS_RAW ',T_OBS_RAW.shape)
    Ge = None
    de = None
    lGe = []
    lde = []
    lcmme = []
    lidx = []
    for i in np.arange(np_names_t_obs.shape[0]):
        CMM = np.eye((T_OBS_RAW.shape[0]))
        code = np_names_t_obs[i]
        print("-- getting Vondrak matrix for %s" % (code) )
        # get index with values
        lindex = np.where( np.isfinite( T_OBS_RAW[:,i,0] )  )[0]
        lidx.append(lindex.size)
        # append A
        #lGe.append( get_A( at.datetime2decyear( at.seconds2datetime(np_obs_date_s[lindex]) ) ) )
        lGe.append( np.eye((lindex.size)) )
        print(lGe[-1].shape)
        # append observation
        lde.append( T_OBS_RAW[lindex,i,0] )
        # common mode parameters
        lcmme.append( CMM[lindex] )


    #print(lde)
    # building the linear system
    print("-- building the linear system")
    G = scipy.linalg.block_diag(*lGe)
    print(G.shape)
    print(lde)
    D = np.hstack(np.array( lde ) )
    print(D.shape)

    # common mode components matrix
    A_CMM = np.vstack( lcmme )
    print(A_CMM.shape)
    # stack
    G = np.hstack((G,A_CMM))

    x, res, rnk, s = scipy.linalg.lstsq(G,D)

    # out Sgts
    out_sgts = copy.deepcopy( Sgts )
    idx = 0
    for i in np.arange(np_names_t_obs.shape[0]):
        code = np_names_t_obs[i]
        out_sgts.__dict__[code].data[:,2] = x[idx:idx+lidx[i]] * 1E-3
        idx += lidx[i]

    out_sgts.append( Gts(code='CMM_') )

    out_sgts.CMM_.data = np.zeros(( T_OBS_RAW.shape[0],10 ))
    out_sgts.CMM_.data[:,0] = at.datetime2decyear( at.seconds2datetime( np_obs_date_s ))
    out_sgts.CMM_.data[:,2] = x[idx:] * 1.E-3



    return out_sgts

In [1]:
from pyacs.gts.Sgts import Sgts
import matplotlib.pyplot as plt
%matplotlib qt

ts_dir = '/Users/nocquet/Dropbox/sse_project/ecuador/pck'
ts = Sgts(ts_dir, verbose=True)
# 'OVSC',
lsite=['POVA','QUIL','AUCA','TEN1','PREC','MAEC','SIEC','GZEC','YTZA',\
       'LUMB','SNLR','TUCO','PASI','COCH','ALEC','PIS1','CXEC','RIOP','GMTE',\
       'CHZO','BAYO']
ts_ref = ts.sub(linclude=lsite).gts('extract_periods',[2014,2015.1])
# gts('detrend_median').
#ts_ref.QUIL.plot()

In [ ]:
nts = common_mode_vondrak(ts_ref,10,verbose=True)


In [ ]:
nts.CMM_.data